In [ ]:
# BART Summarization Experiments
This notebook explores various experiments to evaluate and extend the summarization capabilities of the BART model.

### Objectives:
1. Baseline Summary Generation.
2. Varying Attention Heads.
3. Self-Attention vs Cross-Attention.
4. Generalization Across Datasets.
5. ROUGE Evaluation.
6. Handling Noisy Inputs.
7. Multi-Document Summarization.
8. Extractive vs Abstractive Summarization.
9. Domain Adaptation.
10. Long Document Summarization.

Each experiment is described in detail with its implementation and output.


Experiment 1: Baseline Summary Generation

## Experiment 1: Baseline Summary Generation
**Objective:** Generate a summary using the pre-trained BART model and evaluate its default performance.


In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset

# Load dataset and sample
dataset = load_dataset('cnn_dailymail', '3.0.0')
sample = dataset['test'][0]
input_text = sample['article']
reference_summary = sample['highlights']
print("Sample Article:", input_text[:500])
print("Reference Summary:", reference_summary)

# Baseline summarization function
def bart_baseline_summary(input_text):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

try:
    baseline_summary = bart_baseline_summary(input_text)
    print("Baseline Summary:", baseline_summary)
except Exception as e:
    print(f"Error during baseline summary generation: {e}")


Sample Article: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin
Reference Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Baseline Summary: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed t

Experiment 2: Varying Attention Heads
## Experiment 2: Varying Attention Heads
**Objective:** Test the impact of varying the number of attention heads on the quality of summaries and inference time.


In [2]:
from transformers import BartConfig
import time

def bart_custom_attention_heads(input_text, num_heads):
    config = BartConfig.from_pretrained('facebook/bart-large-cnn')
    config.encoder_attention_heads = num_heads
    config.decoder_attention_heads = num_heads
    model = BartForConditionalGeneration(config)
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

for heads in [4, 8, 16]:
    start_time = time.time()
    try:
        custom_summary = bart_custom_attention_heads(input_text, heads)
        elapsed_time = time.time() - start_time
        print(f"Summary with {heads} attention heads:\n{custom_summary}")
        print(f"Time Taken: {elapsed_time:.2f} seconds\n")
    except Exception as e:
        print(f"Error with {heads} attention heads: {e}")


Summary with 4 attention heads:
FullFullyiographicalographicalographical and and and Produ Produ Produ workshopsFullFullFull reacts intrins intrins intrins Rohingya Rohingya and and intrins intrins draw draw drawographicalographical workshops workshops intrins intrins and and dism dism dismFull workshops workshops workshops dism dism principled principled principledHerHerHer graph graph graph and and470 Rohingya Rohingya emblem emblem emblem graph graph intrins intrinsDetails and andsylvaniasylvaniasylvania dism=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-HerHerederationederationederation draw drawmith and and Aph Aph Aph and and supremacist supremacist intrins intrins graph graph advisory advisoryographicalographical Rohingya Rohingya Rohingya orche orche axis axis axis workshops workshopsographicalographical intrins intrins morphine draw draw 211 211 Local Local Localographicalographicalportionstownstownstown graph graphFullFull grants grants grants elected elected elected intrins

## Experiment 3: Self-Attention vs Cross-Attention
**Objective:** Evaluate the role of self-attention by disabling cross-attention in the decoder.


In [5]:
class BartSelfAttentionOnly(BartForConditionalGeneration):
    def forward(self, *args, **kwargs):
        outputs = super().forward(*args, **kwargs)
        outputs.encoder_attentions = None
        return outputs

def bart_self_attention_summary(input_text):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartSelfAttentionOnly.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

try:
    self_attention_summary = bart_self_attention_summary(input_text)
    print("Self-Attention Summary:", self_attention_summary)
except Exception as e:
    print(f"Error during self-attention summary generation: {e}")


Self-Attention Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Welsh Government said more people than ever were getting help to address housing problems.


# Experiment 4: Generalization Across Datasets

In [3]:
datasets = [
    {'name': 'cnn_dailymail', 'config': '3.0.0'},
    {'name': 'xsum', 'config': None, 'trust_remote_code': True}
]
for dataset_info in datasets:
    dataset_name = dataset_info['name']
    config = dataset_info.get('config', None)
    trust_remote_code = dataset_info.get('trust_remote_code', False)
    try:
        dataset = load_dataset(dataset_name, config, trust_remote_code=trust_remote_code)
        sample = dataset['test'][0]
        input_text = sample['article'] if 'article' in sample else sample['document']
        reference_summary = sample['highlights'] if 'highlights' in sample else sample['summary']
        baseline_summary = bart_baseline_summary(input_text)
        print(f"Dataset: {dataset_name}")
        print("Generated Summary:", baseline_summary)
        print("Reference Summary:", reference_summary)
    except Exception as e:
        print(f"Error with dataset {dataset_name}: {e}")

Dataset: cnn_dailymail
Generated Summary: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
Reference Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Dataset: xsum
Generated Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Welsh Government said more people than ever were getting help to address housing problems.
Reference Summary: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.


# Experiment 5: ROUGE Evaluation

In [9]:
import evaluate
rouge = evaluate.load("rouge")
scores = rouge.compute(predictions=[baseline_summary], references=[reference_summary])
print("ROUGE Scores:", scores)

ROUGE Scores: {'rouge1': 0.24137931034482757, 'rouge2': 0.03571428571428572, 'rougeL': 0.13793103448275862, 'rougeLsum': 0.13793103448275862}


# Experiment 6: Handling Noisy Inputs

In [11]:
import random
def add_noise_to_text(text, noise_level=0.1):
    words = text.split()
    noisy_words = [w if random.random() > noise_level else "randomword" for w in words]
    return " ".join(noisy_words)

noisy_input = add_noise_to_text(input_text)
try:
    noisy_summary = bart_baseline_summary(noisy_input)
    print("Noisy Input Summary:", noisy_summary)
except Exception as e:
    print(f"Error during noisy input summary generation: {e}")

Noisy Input Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up randomword a randomword before finding suitable accommodation. Workers at the charity claim investment in housing would be cheaper than jailing randomword repeat offenders.


# Experiment 7: Multi-Document Summarization

In [12]:
multi_input = input_text + " " + reference_summary
try:
    multi_doc_summary = bart_baseline_summary(multi_input)
    print("Multi-Document Summary:", multi_doc_summary)
except Exception as e:
    print(f"Error during multi-document summary generation: {e}")


Multi-Document Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Welsh Government said more people than ever were getting help to address housing problems.


# Experiment 8: Extractive vs Abstractive Summarization

In [21]:
try:
    print("\n=== Experiment 8: Extractive vs Abstractive Summarization ===")
    
    # Extractive Summarization using BART (adjusted for extractive behavior)
    extractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt")
    extractive_summary = extractive_summarizer(input_text, max_length=50, min_length=10, length_penalty=2.0)
    print("Extractive Summary:", extractive_summary[0]['summary_text'])
    
    # Abstractive Summarization using BART (default behavior)
    abstractive_summary = bart_baseline_summary(input_text)
    print("Abstractive Summary:", abstractive_summary)
except Exception as e:
    print(f"Error during summarization: {e}")




=== Experiment 8: Extractive vs Abstractive Summarization ===
Extractive Summary: The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join.
Abstractive Summary: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.


# Experiment 9: Long Document Summarization

In [26]:
chunks = [input_text[i:i+1024] for i in range(0, len(input_text), 1024)]
chunk_summaries = [bart_baseline_summary(chunk) for chunk in chunks]
final_summary = " ".join(chunk_summaries)
print("Long Document Summary:", final_summary)

Long Document Summary: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.
